In [18]:
from MarkovSwitchingModel import MarkovSwitchingDSGE

In [19]:
nklearn = MarkovSwitchingDSGE.read('3eq_ms.yaml')

In [20]:
nk = nklearn.compile_model()
p0 = nklearn.p0()

In [21]:
from dsge.gensys import gensys
import numpy as np
yy = np.array(nk.yy)
nobs = yy.shape[0]
GAM0 = nk.GAM0(p0)
GAM1 = nk.GAM1(p0)
PSI = nk.PSI(p0)
PPI = nk.PPI(p0)
TT, RR, M, TZ, TY, RC = gensys(GAM0, GAM1, PSI, PPI, return_everything=True)
TT, RR, QQ, DD, ZZ, HH = nk.system_matrices(p0)

initial_distribution = np.array(nk.initial_distribution(p0),dtype=float).squeeze()
filtered_prob = initial_distribution
strans = nk.transition(p0)
sigrq = np.asarray(np.sqrt(nk.conditional_variance(p0))).squeeze()
epsr_m = nk.conditional_mean(p0).squeeze()
ns = TT.shape[0]
neps = RR.shape[1]
ny = ZZ.shape[1]
nj = initial_distribution.size
EXPeffect_f = np.zeros((ns, nj))
EXPeffect_m = np.zeros((ns, nj))

noshock = ~(np.arange(neps)==nk.shock_ind)
Mj = np.eye(M.shape[0])
strans = np.atleast_2d(strans)
stransj = strans
shocksel = np.zeros(neps)
shocksel[~noshock] = 1

filtered_mean_mat = np.zeros((nobs, ns))
filtered_prob_mat = np.zeros((nobs, nj))
epsr_m = np.atleast_1d(np.array(epsr_m))

for j in range(100):
    EXPeffect_f = EXPeffect_f +  (TY @ Mj @ TZ @ shocksel)[:, np.newaxis] @ ( epsr_m.T @ stransj )[np.newaxis, :]
    #EXPeffect_m = EXPeffect_m +  TY @ Mj @ TZ @ shocksel @ ( epsr_m.T )

    Mj = Mj @ M
    stransj = stransj @ strans
EXPeffect_f = np.real(EXPeffect_f)

In [23]:
EXPeffect_f.round(3)

array([[ 0.067,  0.111, -0.   , -0.111, -0.067],
       [ 0.025,  0.043, -0.   , -0.043, -0.025],
       [ 0.042,  0.071, -0.   , -0.071, -0.042],
       [-0.   , -0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   , -0.   , -0.   , -0.   ],
       [-0.   , -0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.111,  0.183, -0.   , -0.183, -0.111],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.037,  0.064, -0.   , -0.064, -0.037]])

In [24]:
nk.state_names

['y', 'R', 'dp', 'e', 'z', 'ylag', 'y(1)', 'z(1)', 'dp(1)']